In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ["TORCH_CUDA_ARCH_LIST"] = "sm_86"
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Capstone')

# library
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.cluster import KMeans
import sklearn
from torch.utils.data import Dataset, DataLoader
from tqdm import trange, tqdm
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from model_transformer5 import TSTransformerEncoder
from torch.optim.optimizer import Optimizer
from collections import OrderedDict
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from tae import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# dataset load
# 전기
elec_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/elec_clustering_train.csv', index_col=0)
elec_train.index = pd.to_datetime(elec_train.index)
elec_train = elec_train.resample(rule='12H').sum()
elec_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/elec_clustering_test.csv', index_col=0)
elec_test.index = pd.to_datetime(elec_test.index)
elec_test = elec_test.resample(rule='12H').sum()
elec_total = pd.concat([elec_train,elec_test],axis=1)
# 수도
water_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/water_clustering_train.csv', index_col=0)
water_train.index = pd.to_datetime(water_train.index)
water_train = water_train.resample(rule='12H').sum()
water_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/water_clustering_test.csv', index_col=0)
water_test.index = pd.to_datetime(water_test.index)
water_test = water_test.resample(rule='12H').sum()
water_total = pd.concat([water_train,water_test],axis=1)
#가스
gas_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/gas_clustering_train.csv', index_col=0)
gas_train.index = pd.to_datetime(gas_train.index)
gas_train = gas_train.resample(rule='12H').sum()
gas_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/gas_clustering_test.csv', index_col=0)
gas_test.index = pd.to_datetime(gas_test.index)
gas_test = gas_test.resample(rule='12H').sum()
gas_total = pd.concat([gas_train,gas_test],axis=1)
#온수
hotwater_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/hotwater_clustering_train.csv', index_col=0)
hotwater_train.index = pd.to_datetime(hotwater_train.index)
hotwater_train = hotwater_train.resample(rule='12H').sum()
hotwater_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/data/hotwater_clustering_test.csv', index_col=0)
hotwater_test.index = pd.to_datetime(hotwater_test.index)
hotwater_test = hotwater_test.resample(rule='12H').sum()
hotwater_total = pd.concat([hotwater_train,hotwater_test],axis=1)

region_A = elec_train.columns
region_B = elec_test.columns

train_index = 504*2
test_index = (504+28)*2
cluster_index = (504+28+28)*2

energy_A = elec_total
energy_B = water_total
energy_C = hotwater_total
energy_target = gas_total

train_A = energy_A[:train_index]
train_B = energy_B[:train_index]
train_C = energy_C[:train_index]
test_A = energy_A[train_index:test_index]
test_B = energy_B[train_index:test_index]
test_C = energy_C[train_index:test_index]
test_target = energy_target[train_index:test_index]
cluster_A = energy_A[test_index:cluster_index]
cluster_B = energy_B[test_index:cluster_index]
cluster_C = energy_C[test_index:cluster_index]
cluster_target = energy_target[test_index:cluster_index]

scaler_A = MinMaxScaler()
scaler_B = MinMaxScaler()
scaler_C = MinMaxScaler()
train_A = pd.DataFrame(scaler_A.fit_transform(train_A))
train_B = pd.DataFrame(scaler_B.fit_transform(train_B))
train_C = pd.DataFrame(scaler_C.fit_transform(train_C))
test_A = pd.DataFrame(scaler_A.transform(test_A))
test_B = pd.DataFrame(scaler_B.transform(test_B))
test_C = pd.DataFrame(scaler_C.transform(test_C))
cluster_A = pd.DataFrame(scaler_A.transform(cluster_A))
cluster_B = pd.DataFrame(scaler_B.transform(cluster_B))
cluster_C = pd.DataFrame(scaler_C.transform(cluster_C))

# 2-5
repr1_A = energy_A[train_index+14:test_index+14]
repr1_B = energy_B[train_index+14:test_index+14]
repr1_C = energy_C[train_index+14:test_index+14]
repr1_A = pd.DataFrame(scaler_A.transform(repr1_A))
repr1_B = pd.DataFrame(scaler_B.transform(repr1_B))
repr1_C = pd.DataFrame(scaler_C.transform(repr1_C))

# 3-6
repr2_A = energy_A[train_index+28:test_index+28]
repr2_B = energy_B[train_index+28:test_index+28]
repr2_C = energy_C[train_index+28:test_index+28]
repr2_A = pd.DataFrame(scaler_A.transform(repr2_A))
repr2_B = pd.DataFrame(scaler_B.transform(repr2_B))
repr2_C = pd.DataFrame(scaler_C.transform(repr2_C))

# 4-7
repr3_A = energy_A[train_index+42:test_index+42]
repr3_B = energy_B[train_index+42:test_index+42]
repr3_C = energy_C[train_index+42:test_index+42]
repr3_A = pd.DataFrame(scaler_A.transform(repr3_A))
repr3_B = pd.DataFrame(scaler_B.transform(repr3_B))
repr3_C = pd.DataFrame(scaler_C.transform(repr3_C))

data_train = []
for user in range(elec_total.shape[1]):
    for index in range(0,elec_total.shape[0]//(28*2)-2):
        window = []
        for period in range(index*28*2,(index+1)*28*2):
            window.append([train_A.iloc[period,user],train_B.iloc[period,user],train_C.iloc[period,user]])
        data_train.append(window)

data_test = []
for user in range(elec_total.shape[1]):
    for index in range(0,1):
        window = []
        for period in range(index*28*2,(index+1)*28*2):
            window.append([test_A.iloc[period,user],test_B.iloc[period,user],test_C.iloc[period,user]])
        data_test.append(window)

data_repr1 = []
for user in range(elec_total.shape[1]):
    for index in range(0,1):
        window = []
        for period in range(index*28*2,(index+1)*28*2):
            window.append([repr1_A.iloc[period,user],repr1_B.iloc[period,user],repr1_C.iloc[period,user]])
        data_repr1.append(window)

data_repr2 = []
for user in range(elec_total.shape[1]):
    for index in range(0,1):
        window = []
        for period in range(index*28*2,(index+1)*28*2):
            window.append([repr2_A.iloc[period,user],repr2_B.iloc[period,user],repr2_C.iloc[period,user]])
        data_repr2.append(window)

data_repr3 = []
for user in range(elec_total.shape[1]):
    for index in range(0,1):
        window = []
        for period in range(index*28*2,(index+1)*28*2):
            window.append([repr3_A.iloc[period,user],repr3_B.iloc[period,user],repr3_C.iloc[period,user]])
        data_repr3.append(window)

# dataset
data_train = np.array(data_train)
data_test = np.array(data_test)
data_repr1 = np.array(data_repr1)
data_repr2 = np.array(data_repr2)
data_repr3 = np.array(data_repr3)

# masking
data_train_mask = []
for i in range(data_train.shape[0]):
    data_train_mask.append(noise_mask(data_train[i]))

data_test_mask = []
for i in range(data_test.shape[0]):
    data_test_mask.append(noise_mask(data_test[i]))

data_repr1_mask = []
for i in range(data_repr1.shape[0]):
    data_repr1_mask.append(noise_mask(data_repr1[i]))

data_repr2_mask = []
for i in range(data_repr2.shape[0]):
    data_repr2_mask.append(noise_mask(data_repr2[i]))

data_repr3_mask = []
for i in range(data_repr3.shape[0]):
    data_repr3_mask.append(noise_mask(data_repr3[i]))

data_train_mask = np.array(data_train_mask)
data_test_mask = np.array(data_test_mask)
data_repr1_mask = np.array(data_repr1_mask)
data_repr2_mask = np.array(data_repr2_mask)
data_repr3_mask = np.array(data_repr3_mask)

# window dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, mask):
        self.X = X
        self.target = X
        self.mask = mask
        # self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.target[index], self.mask[index]

ds_train = TimeSeriesDataset(data_train, data_train_mask)
ds_test = TimeSeriesDataset(data_test, data_test_mask)
ds_repr1 = TimeSeriesDataset(data_repr1, data_repr1_mask)
ds_repr2 = TimeSeriesDataset(data_repr2, data_repr2_mask)
ds_repr3 = TimeSeriesDataset(data_repr3, data_repr3_mask)

batch_size = 128
train_loader = DataLoader(dataset=ds_train, batch_size=batch_size, drop_last=True, shuffle=True)
test_loader = DataLoader(dataset=ds_test, batch_size=batch_size, drop_last=False, shuffle=False)
repr1_loader = DataLoader(dataset=ds_repr1, batch_size=batch_size, drop_last=False, shuffle=False)
repr2_loader = DataLoader(dataset=ds_repr2, batch_size=batch_size, drop_last=False, shuffle=False)
repr3_loader = DataLoader(dataset=ds_repr3, batch_size=batch_size, drop_last=False, shuffle=False)

In [3]:
num = 0
device = torch.device('cuda')
model = TSTransformerEncoder(feat_dim=data_train.shape[2], max_len=data_train.shape[1], d_model=64, n_heads=8, num_layers=1, dim_feedforward=256, pos_encoding='learnable')
model.to(device)
# optimizer = RAdam(model.parameters())
optimizer = torch.optim.AdamW(model.parameters())

# training loop
model.train()
num_epochs = 200
loss_module = MaskedMSELoss()
loss_list = []
val_list = []
valmask_list = []
val_output = []
val_true = []
criterion = nn.MSELoss()
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0  # total loss of epoch
    total_active_elements = 0  # total unmasked elements in epoch
    for i, batch in enumerate(train_loader):

        X, targets, target_masks = batch # X는 mask되지 않은 값
        target_masks = target_masks.to(device) # 1s: mask and predict, 0s: unaffected input (ignore) // noise_mask 에서 옴
        X = X.to(device)
        targets = torch.tensor(targets.to(device), dtype = torch.float32) # mask 되지 않은 값
        X = X * target_masks # mask 된 input
        X = torch.tensor(X, dtype = torch.float32)
        predictions, _ = model(X)  # (batch_size, padded_length, feat_dim)

        # Cascade noise masks (batch_size, padded_length, feat_dim) and padding masks (batch_size, padded_length)
        loss = loss_module(predictions, targets, ~target_masks)  # ~target_mask를 prediction, target에 역으로 곱해서 mask한 값만 loss 계산함
        total_loss = loss
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
        optimizer.step()
        total_active_elements += 1
        epoch_loss += total_loss.item()
    with torch.no_grad():
        val_loss = 0.0
        valmask_loss = 0.0
        for i, batch in enumerate(test_loader):
            X, targets, target_masks = batch # X는 mask되지 않은 값
            target_masks = target_masks.to(device) # 1s: mask and predict, 0s: unaffected input (ignore) // noise_mask 에서 옴
            X = X.to(device)
            targets = torch.tensor(targets.to(device), dtype = torch.float32) # mask 되지 않은 값
            X = X * target_masks # mask 된 input
            X = torch.tensor(X, dtype = torch.float32)
            predictions, _ = model(X)
            val_loss += criterion(predictions, targets.float()).item()
            valmask_loss += loss_module(predictions, targets, ~target_masks)
        val_list.append(val_loss)
        valmask_list.append(valmask_loss)
        if epoch == num_epochs-1:
            val_true.append(targets.cpu())
            val_output.append(predictions.cpu())
    epoch_loss = epoch_loss / total_active_elements
    loss_list.append(epoch_loss)
print(epoch_loss)


repr1_hidden = []
with torch.no_grad():
    for i, batch in enumerate(repr1_loader):
        X, _, _ = batch # X는 mask되지 않은 값
        X = X.to(device)
        X = torch.tensor(X, dtype = torch.float32)
        _, hidden = model(X)
        repr1_hidden.append(hidden.cpu().numpy())
arr_hidden_1 = []
for i in range(9):
    for j in range(repr1_hidden[i].shape[0]):
        arr_hidden_1.append(repr1_hidden[i][j])
arr_hidden_1 = pd.DataFrame(np.reshape(arr_hidden_1,(1080,64*28*2)))

repr2_hidden = []
with torch.no_grad():
    for i, batch in enumerate(repr2_loader):
        X, _, _ = batch # X는 mask되지 않은 값
        X = X.to(device)
        X = torch.tensor(X, dtype = torch.float32)
        _, hidden = model(X)
        repr2_hidden.append(hidden.cpu().numpy())
arr_hidden_2 = []
for i in range(9):
    for j in range(repr2_hidden[i].shape[0]):
        arr_hidden_2.append(repr2_hidden[i][j])
arr_hidden_2 = pd.DataFrame(np.reshape(arr_hidden_2,(1080,64*28*2)))

repr3_hidden = []
with torch.no_grad():
    for i, batch in enumerate(repr3_loader):
        X, _, _ = batch # X는 mask되지 않은 값
        X = X.to(device)
        X = torch.tensor(X, dtype = torch.float32)
        _, hidden = model(X)
        repr3_hidden.append(hidden.cpu().numpy())
arr_hidden_3 = []
for i in range(9):
    for j in range(repr3_hidden[i].shape[0]):
        arr_hidden_3.append(repr3_hidden[i][j])
arr_hidden_3 = pd.DataFrame(np.reshape(arr_hidden_3,(1080,64*28*2)))


arr_hidden_1.index = elec_total.columns
arr_hidden_1.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/result/tae_gas/arr_hidden_{}_{}.csv'.format(1, num))
arr_hidden_2.index = elec_total.columns
arr_hidden_2.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/result/tae_gas/arr_hidden_{}_{}.csv'.format(2, num))
arr_hidden_3.index = elec_total.columns
arr_hidden_3.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/result/tae_gas/arr_hidden_{}_{}.csv'.format(3, num))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
  0%|          | 0/200 [00:00<?, ?it/s]<ipython-input-3-cdf7f5c9cc94>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets.to(device), dtype = torch.float32) # mask 되지 않은 값
<ipython-input-3-cdf7f5c9cc94>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype = torch.float32)
<ipython-input-3-cdf7f5c9cc94>:49: Use

0.014632497877129261
